# Capstone Project - The Battle of Neighborhoods

### Introduction: 

Background New York City (NYC) is considered as one of the most populous city in the United Stated. The City comprises of five Boroughs with an estimate population of 8 million people distributed over the areas. New York is known as the City that is populated by people of various ethnic groups. Indians in New York City metropolitan region constitute one of the largest and fastest growing ethnicities in the New York City metropolitan area of the United States. The New York City region is home to the largest Indian American population among metropolitan areas by a significant margin. With the density of population and high level of business activities in New York, NYC is one of the cities that have high potential for start up of any new business venture.


A person would like to open a new business venture. It can be a grocery store, office place or someone starting their own business. What would be the best possible place from where one can start off.

So, in this project we will try to find the solution of above mentioned questions and try to find out best places in New York for Indian Cuisine along with the best borough where a person can start their own business.


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [82]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Lets load and explore the data

In [83]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [84]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [85]:
json_data = newyork_data['features']

In [86]:
json_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Tranform the data into a pandas dataframe

In [87]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
nydf = pd.DataFrame(columns=column_names)
nydf.head()

,Borough,Neighborhood,Latitude,Longitude


Let's loop through the data and fill the dataframe one row at a time.

In [88]:
for data in json_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    nydf = nydf.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [89]:
nydf.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Use geopy library to get the latitude and longitude values of New York City

In [90]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


## Create a map of New York with neighborhoods superimposed on top

In [91]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nydf['Latitude'], nydf['Longitude'], nydf['Borough'], nydf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare Credentials and Version

In [92]:
CLIENT_ID = 'Y2MQTFDXRP25NUSFS4XT5EPM3RWHVVDXQURS4GARWWERRBP0' # your Foursquare ID
CLIENT_SECRET = 'HLGAWVJ43MCVLWTTPFXNLVVR1PDT3FEXPGFOSNUJ01F2FSKV' # your Foursquare Secret
VERSION = '20190605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y2MQTFDXRP25NUSFS4XT5EPM3RWHVVDXQURS4GARWWERRBP0
CLIENT_SECRET:HLGAWVJ43MCVLWTTPFXNLVVR1PDT3FEXPGFOSNUJ01F2FSKV


In [93]:
nydf.loc[2, 'Neighborhood']

'Eastchester'

Explore Neighborhoods in New york and creating a function to repeat the same process to all the neighborhoods in Toronto

In [94]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT = 100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighburhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Write a code to run the above function on each neighborhood and create a new dataframe called ny_venues

In [95]:
ny_venues = getNearbyVenues(names=nydf['Neighborhood'],
                                   latitudes=nydf['Latitude'],
                                   longitudes=nydf['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [96]:
print(ny_venues.shape)
ny_venues.head()

(10396, 7)


,Neighborhood,Neighborhood Latitude,Neighburhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


Checking how many venues were returned for each neighborhood

In [97]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighburhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,31,31,31,31,31,31
Annadale,9,9,9,9,9,9
Arden Heights,4,4,4,4,4,4
Arlington,6,6,6,6,6,6
Arrochar,20,20,20,20,20,20
Arverne,17,17,17,17,17,17
Astoria,100,100,100,100,100,100
Astoria Heights,14,14,14,14,14,14
Auburndale,21,21,21,21,21,21


Let's find out how many unique categories can be curated from all the returned venues

In [98]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 432 uniques categories.


Analyze Each Neighborhood

In [99]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Orga

In [100]:
ny_onehot.shape

(10396, 432)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [101]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Orga

In [102]:
ny_grouped.shape

(302, 432)

print each neighborhood along with the top 5 most common venues

In [103]:
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
           venue  freq
0    Pizza Place  0.13
1  Deli / Bodega  0.06
2    Supermarket  0.06
3            Spa  0.06
4       Pharmacy  0.06


----Annadale----
           venue  freq
0    Pizza Place  0.22
1     Restaurant  0.11
2          Diner  0.11
3       Pharmacy  0.11
4  Train Station  0.11


----Arden Heights----
         venue  freq
0  Pizza Place  0.25
1  Coffee Shop  0.25
2     Bus Stop  0.25
3     Pharmacy  0.25
4  Yoga Studio  0.00


----Arlington----
                 venue  freq
0        Deli / Bodega  0.17
1                 Food  0.17
2  American Restaurant  0.17
3        Boat or Ferry  0.17
4             Bus Stop  0.17


----Arrochar----
                venue  freq
0       Deli / Bodega  0.15
1            Bus Stop  0.10
2         Pizza Place  0.10
3  Italian Restaurant  0.10
4         Supermarket  0.05


----Arverne----
             venue  freq
0        Surf Spot  0.24
1    Metro Station  0.12
2            Beach  0.06
3  Thai Restaurant  0.06
4       Playgr

putting data into a pandas dataframe writing a function to sort the venues in descending order.

In [104]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

creating a new dataframe and display the top 10 venues for each neighborhood.

In [105]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Supermarket,Spa,Chinese Restaurant,Pharmacy,Playground,Bus Station,Spanish Restaurant,Breakfast Spot
1,Annadale,Pizza Place,Restaurant,Sports Bar,Train Station,Food,Diner,Pharmacy,Park,Fast Food Restaurant,Event Space
2,Arden Heights,Pharmacy,Pizza Place,Coffee Shop,Bus Stop,Women's Store,Financial or Legal Service,Eye Doctor,Factory,Falafel Restaurant,Farm
3,Arlington,Food,Deli / Bodega,American Restaurant,Boat or Ferry,Grocery Store,Bus Stop,Women's Store,Financial or Legal Service,Fish Market,Fish & Chips Shop
4,Arrochar,Deli / Bodega,Italian Restaurant,Bus Stop,Pizza Place,Outdoors & Recreation,Mediterranean Restaurant,Middle Eastern Restaurant,Bagel Shop,Liquor Store,Sandwich Place


Cluster Neighborhoods k-means to cluster the neighborhood into 3 clusters

In [114]:
# set number of clusters
kclusters = 3

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [115]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0,'Cluster Labels',  kmeans.labels_)

ny_merged = nydf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1,Caribbean Restaurant,Donut Shop,Pharmacy,Gas Station,Dessert Shop,Food Truck,Sandwich Place,Ice Cream Shop,Laundromat,Field
1,Bronx,Co-op City,40.874294,-73.829939,1,Baseball Field,Restaurant,Bus Station,Pharmacy,Ice Cream Shop,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Grocery Store,Park
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Bus Station,Deli / Bodega,Diner,Bus Stop,Metro Station,Donut Shop,Food & Drink Shop,Automotive Shop,Pizza Place
3,Bronx,Fieldston,40.895437,-73.905643,1,River,High School,Bus Station,Playground,Plaza,Financial or Legal Service,Exhibit,Eye Doctor,Factory,Falafel Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,0,Bus Station,Park,Bank,Food Truck,Home Service,Plaza,Gym,Financial or Legal Service,Eye Doctor,Factory


visualizing the resulting clusters

In [116]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Exmaine the clusters

It can be seen that cluster 0 indicates venues or places like Parks, Bus station as most common for Borough’s like Bronx, Queens and the neighborhoods around it. Cluster 0 consists of venues or places like Parks, Bus station as most common for Borough’s like Bronx, Queens and the neighborhoods around it and hence based on results it can be said ideal for starting any new business venture as initial set up investment would be less as the cluster is not so densely populated with markets. 

In [117]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Bronx,Riverdale,40.890834,-73.912585,0,Bus Station,Park,Bank,Food Truck,Home Service,Plaza,Gym,Financial or Legal Service,Eye Doctor,Factory
27,Bronx,Clason Point,40.806551,-73.854144,0,Park,Pool,Scenic Lookout,Bus Stop,Business Service,Boat or Ferry,Grocery Store,South American Restaurant,Factory,Falafel Restaurant
35,Bronx,Spuyten Duyvil,40.881395,-73.917190,0,Park,Bank,Waste Facility,Thai Restaurant,Tennis Stadium,Scenic Lookout,Pharmacy,Bus Line,Intersection,Food Court
148,Queens,South Ozone Park,40.668550,-73.809865,0,Deli / Bodega,Park,Bar,Hotel,Sandwich Place,Gym / Fitness Center,Fast Food Restaurant,Donut Shop,Food Truck,Farm
171,Queens,Broad Channel,40.603027,-73.820055,0,Deli / Bodega,Pizza Place,Bus Station,Other Nightlife,Dive Bar,Park,Women's Store,Fish Market,Falafel Restaurant,Farm
192,Queens,Somerville,40.597711,-73.796648,0,Park,Women's Store,Fish Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
193,Queens,Brookville,40.660003,-73.751753,0,Deli / Bodega,Women's Store,Flea Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
198,Staten Island,New Brighton,40.640615,-74.087017,0,Bus Stop,Deli / Bodega,Park,Bowling Alley,Playground,Convenience Store,Discount Store,Filipino Restaurant,Fish & Chips Shop,Financial or Legal Service
203,Staten Island,Todt Hill,40.597069,-74.111329,0,Park,Women's Store,Fish Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
245,Staten Island,Bloomfield,40.605779,-74.187256,0,Theme Park,Recreation Center,Discount Store,Bus Stop,Park,Women's Store,Fish Market,Falafel Restaurant,Farm,Farmers Market


## Examining the clusters.
 The  cluster 1 lists out places like Restaurants, coffee shops, bars and pubs as the most common venues. In a nutshell it gives us all the places where people can have a quick snack or any eatable item that they wish to eat.

 The Borough and neighborhoods involved can be good for someone looking to open any restaurants, coffee shops or any shops for eateries.


In [118]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1,Caribbean Restaurant,Donut Shop,Pharmacy,Gas Station,Dessert Shop,Food Truck,Sandwich Place,Ice Cream Shop,Laundromat,Field
1,Bronx,Co-op City,40.874294,-73.829939,1,Baseball Field,Restaurant,Bus Station,Pharmacy,Ice Cream Shop,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Grocery Store,Park
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Bus Station,Deli / Bodega,Diner,Bus Stop,Metro Station,Donut Shop,Food & Drink Shop,Automotive Shop,Pizza Place
3,Bronx,Fieldston,40.895437,-73.905643,1,River,High School,Bus Station,Playground,Plaza,Financial or Legal Service,Exhibit,Eye Doctor,Factory,Falafel Restaurant
5,Bronx,Kingsbridge,40.881687,-73.902818,1,Pizza Place,Sandwich Place,Deli / Bodega,Discount Store,Bar,Latin American Restaurant,Supermarket,Mexican Restaurant,Pharmacy,Donut Shop
6,Manhattan,Marble Hill,40.876551,-73.910660,1,Coffee Shop,Discount Store,Sandwich Place,Yoga Studio,Donut Shop,Spa,Steakhouse,Supplement Shop,Gym,Tennis Stadium
7,Bronx,Woodlawn,40.898273,-73.867315,1,Pub,Playground,Deli / Bodega,Food & Drink Shop,Pizza Place,Rental Car Location,Laundromat,Bus Stop,Food Truck,Supermarket
8,Bronx,Norwood,40.877224,-73.879391,1,Pizza Place,Deli / Bodega,Park,Bank,Pharmacy,Convenience Store,Sandwich Place,Liquor Store,Caribbean Restaurant,Mobile Phone Shop
9,Bronx,Williamsbridge,40.881039,-73.857446,1,Bar,Nightclub,Caribbean Restaurant,Soup Place,Fish Market,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
10,Bronx,Baychester,40.866858,-73.835798,1,Pet Store,Donut Shop,Electronics Store,BBQ Joint,Baseball Field,Sandwich Place,Pizza Place,Mexican Restaurant,Bus Station,Breakfast Spot


The cluster 2 is shown below


In [119]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
207,Staten Island,Port Ivory,40.639683,-74.174645,2,Bar,Women's Store,Fish Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
